# Retrieval Evaluation

In [1]:
import os
# Setting temp dir to /srv/data directory, as it otherwise fills up the home directory too much
# Just comment out on machines that are not "Goober"
os.environ["TMPDIR"] = "/srv/data/tmp"
os.makedirs("/srv/data/tmp", exist_ok=True)

%pip install haystack-ai
%pip install ragas-haystack
%pip install nltk
%pip install openai
%pip install pandas
%pip install ragas-haystack
%pip install sentence-transformers>=3.0.0
%pip install hf_xet
%pip install ollama-haystack==2.4.2
%pip install tqdm # For Progress Bar

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import pandas as pd
from haystack.document_stores.in_memory import InMemoryDocumentStore
import importlib
from haystack.components.evaluators import DocumentMRREvaluator, DocumentMAPEvaluator, DocumentRecallEvaluator
from tqdm import tqdm
tqdm.pandas()
import logging

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s %(levelname)s %(message)s',
    datefmt='%H:%M:%S'
)

In [2]:
from config.secret import OPENAI_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

os.environ["EMBEDDING_MODEL_NAME"] = "Qwen/Qwen3-Embedding-4B"
os.environ["SENTENCE_TRANSFORMERS_HOME"] = "./model-assets/sentence-transformers"
os.environ["EMBEDDING_MODEL_TOP_K"] = "10"

os.environ["LLM_NAME"] = "gemma3:12b"
os.environ["LLM_CONTEXT_SIZE"] = "8192"

In [ ]:
df = pd.read_pickle("data/qa_with_docs_flat/question_answers_docs_line_1_0_dataset_flat.pkl")
all_documents = pd.read_pickle("data/all_documents.pkl")["document"].tolist()

In [5]:
print(df.head())

                                            question  \
0  Wie läuft eine äußere Wendung ab und was passi...   
1  Und wie lange dauert dieser Eingriff normalerw...   
2  Ey, wie machen die eigentlich so 'ne äußere We...   
3  How is an external version done and what happe...   
4  Können Sie mir erklären, wie eine äußere Wendu...   

                                          references  \
0  [Eine äußere Wendung findet in der Regel in ei...   
1  [Eine äußere Wendung findet in der Regel in ei...   
2  [Eine äußere Wendung findet in der Regel in ei...   
3  [Eine äußere Wendung findet in der Regel in ei...   
4  [Eine äußere Wendung findet in der Regel in ei...   

                                         groundTruth         source_file  \
0  Eine äußere Wendung findet ambulant in einer K...  Äußere Wendung.md   
1  Eine äußere Wendung findet ambulant in einer K...  Äußere Wendung.md   
2  Eine äußere Wendung findet ambulant in einer K...  Äußere Wendung.md   
3  Eine äußere Wendung

## Setup Retrieval Pipelines

Sets up the basic / hybrid RAG pipelines. These can then be used later in combintation with the different indexing methods from before to evaluate different RAG setups.

In [6]:
base_indexing_store = InMemoryDocumentStore.load_from_disk(f"data/document_stores/{os.environ['EMBEDDING_MODEL_NAME']}/base_indexing_store.json")
context_indexing_store = InMemoryDocumentStore.load_from_disk(f"data/document_stores/{os.environ['EMBEDDING_MODEL_NAME']}/context_indexing_store.json")

In [56]:
context_indexing_store.filter_documents()

[]

In [55]:
base_indexing_store.filter_documents()

[Document(id=cd20e2851eabc57193d0701ee1feb0bb91b1baea70e6921324ad565fd656bee1, content: 'Weshalb wird eine äußere Wendung in Erwägung gezogen?
 
 ', meta: {'title': 'Äußere Wendung.md', 'source_id': '7ae1e3f0-dd60-4371-ab7b-aeeebef45f64', 'page_number': 1, 'split_id': 0, 'split_idx_start': 0}, embedding: vector of size 2560),
 Document(id=064cc49648b2da93bbbdf3c5eff49d64d533c6036a3a6345ac652f9cbd710884, content: 'Im Laufe der Schwangerschaft kann das ungeborene Kind seine Position im Mutterleib mehrfach veränder...', meta: {'title': 'Äußere Wendung.md', 'source_id': '7ae1e3f0-dd60-4371-ab7b-aeeebef45f64', 'page_number': 1, 'split_id': 1, 'split_idx_start': 55}, embedding: vector of size 2560),
 Document(id=8cf370d93ab8a2d239a9733288e705d0f44ac146396abb75955aa908aff7694b, content: 'Verbleibt das Kind hingegen am Schwangerschaftsende in einer anderen Haltung – etwa mit dem Becken n...', meta: {'title': 'Äußere Wendung.md', 'source_id': '7ae1e3f0-dd60-4371-ab7b-aeeebef45f64', 'page_num

In [43]:
import pipelines.retrieval_pipelines.base_retrieval_pipeline
importlib.reload(pipelines.retrieval_pipelines.base_retrieval_pipeline)
from pipelines.components.wrap_in_list_adapter import WrapInListAdapter

base_retrieval_pipeline = pipelines.retrieval_pipelines.base_retrieval_pipeline.get_base_retrieval_pipeline(base_indexing_store)

map_evaluator = DocumentMAPEvaluator()
mrr_evaluator = DocumentMRREvaluator()
recall_evaluator = DocumentRecallEvaluator()

retriever_to_evaluator_adapter = WrapInListAdapter()

base_retrieval_pipeline.add_component(instance=map_evaluator, name="map_evaluator")
base_retrieval_pipeline.add_component(instance=mrr_evaluator, name="mrr_evaluator")
base_retrieval_pipeline.add_component(instance=recall_evaluator, name="recall_evaluator")
base_retrieval_pipeline.add_component(instance=retriever_to_evaluator_adapter, name="retriever_to_evaluator_adapter")
base_retrieval_pipeline.connect("retriever", "retriever_to_evaluator_adapter")
base_retrieval_pipeline.connect("retriever_to_evaluator_adapter.output", "map_evaluator.retrieved_documents")
base_retrieval_pipeline.connect("retriever_to_evaluator_adapter.output", "mrr_evaluator.retrieved_documents")
base_retrieval_pipeline.connect("retriever_to_evaluator_adapter.output", "recall_evaluator.retrieved_documents")

🚅 Components
  - query_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - map_evaluator: DocumentMAPEvaluator
  - mrr_evaluator: DocumentMRREvaluator
  - recall_evaluator: DocumentRecallEvaluator
  - retriever_to_evaluator_adapter: WrapInListAdapter
🛤️ Connections
  - query_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> retriever_to_evaluator_adapter.documents (List[Document])
  - retriever_to_evaluator_adapter.output -> map_evaluator.retrieved_documents (List[List[Document]])
  - retriever_to_evaluator_adapter.output -> mrr_evaluator.retrieved_documents (List[List[Document]])
  - retriever_to_evaluator_adapter.output -> recall_evaluator.retrieved_documents (List[List[Document]])

In [44]:
import pipelines.retrieval_pipelines.hybrid_retrieval_pipeline
importlib.reload(pipelines.retrieval_pipelines.hybrid_retrieval_pipeline)

hybrid_retrieval_pipeline = pipelines.retrieval_pipelines.hybrid_retrieval_pipeline.get_hybrid_retrieval_pipeline(base_indexing_store)

map_evaluator = DocumentMAPEvaluator()
mrr_evaluator = DocumentMRREvaluator()
recall_evaluator = DocumentRecallEvaluator()

retriever_to_evaluator_adapter = WrapInListAdapter()

hybrid_retrieval_pipeline.add_component(instance=map_evaluator, name="map_evaluator")
hybrid_retrieval_pipeline.add_component(instance=mrr_evaluator, name="mrr_evaluator")
hybrid_retrieval_pipeline.add_component(instance=recall_evaluator, name="recall_evaluator")
hybrid_retrieval_pipeline.add_component(instance=retriever_to_evaluator_adapter, name="retriever_to_evaluator_adapter")
hybrid_retrieval_pipeline.connect("joiner", "retriever_to_evaluator_adapter")
hybrid_retrieval_pipeline.connect("retriever_to_evaluator_adapter.output", "map_evaluator.retrieved_documents")
hybrid_retrieval_pipeline.connect("retriever_to_evaluator_adapter.output", "mrr_evaluator.retrieved_documents")
hybrid_retrieval_pipeline.connect("retriever_to_evaluator_adapter.output", "recall_evaluator.retrieved_documents")

🚅 Components
  - query_embedder: SentenceTransformersTextEmbedder
  - bm25_retriever: InMemoryBM25Retriever
  - retriever: InMemoryEmbeddingRetriever
  - joiner: DocumentJoiner
  - map_evaluator: DocumentMAPEvaluator
  - mrr_evaluator: DocumentMRREvaluator
  - recall_evaluator: DocumentRecallEvaluator
  - retriever_to_evaluator_adapter: WrapInListAdapter
🛤️ Connections
  - query_embedder.embedding -> retriever.query_embedding (List[float])
  - bm25_retriever.documents -> joiner.documents (List[Document])
  - retriever.documents -> joiner.documents (List[Document])
  - joiner.documents -> retriever_to_evaluator_adapter.documents (List[Document])
  - retriever_to_evaluator_adapter.output -> map_evaluator.retrieved_documents (List[List[Document]])
  - retriever_to_evaluator_adapter.output -> mrr_evaluator.retrieved_documents (List[List[Document]])
  - retriever_to_evaluator_adapter.output -> recall_evaluator.retrieved_documents (List[List[Document]])

In [45]:
import pipelines.retrieval_pipelines.base_retrieval_pipeline
importlib.reload(pipelines.retrieval_pipelines.base_retrieval_pipeline)

context_retrieval_pipeline = pipelines.retrieval_pipelines.base_retrieval_pipeline.get_base_retrieval_pipeline(context_indexing_store)

map_evaluator = DocumentMAPEvaluator()
mrr_evaluator = DocumentMRREvaluator()
recall_evaluator = DocumentRecallEvaluator()

retriever_to_evaluator_adapter = WrapInListAdapter()

context_retrieval_pipeline.add_component(instance=map_evaluator, name="map_evaluator")
context_retrieval_pipeline.add_component(instance=mrr_evaluator, name="mrr_evaluator")
context_retrieval_pipeline.add_component(instance=recall_evaluator, name="recall_evaluator")
context_retrieval_pipeline.add_component(instance=retriever_to_evaluator_adapter, name="retriever_to_evaluator_adapter")
context_retrieval_pipeline.connect("retriever", "retriever_to_evaluator_adapter")
context_retrieval_pipeline.connect("retriever_to_evaluator_adapter.output", "map_evaluator.retrieved_documents")
context_retrieval_pipeline.connect("retriever_to_evaluator_adapter.output", "mrr_evaluator.retrieved_documents")
context_retrieval_pipeline.connect("retriever_to_evaluator_adapter.output", "recall_evaluator.retrieved_documents")

🚅 Components
  - query_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - map_evaluator: DocumentMAPEvaluator
  - mrr_evaluator: DocumentMRREvaluator
  - recall_evaluator: DocumentRecallEvaluator
  - retriever_to_evaluator_adapter: WrapInListAdapter
🛤️ Connections
  - query_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> retriever_to_evaluator_adapter.documents (List[Document])
  - retriever_to_evaluator_adapter.output -> map_evaluator.retrieved_documents (List[List[Document]])
  - retriever_to_evaluator_adapter.output -> mrr_evaluator.retrieved_documents (List[List[Document]])
  - retriever_to_evaluator_adapter.output -> recall_evaluator.retrieved_documents (List[List[Document]])

In [46]:
import pipelines.retrieval_pipelines.hybrid_retrieval_pipeline
importlib.reload(pipelines.retrieval_pipelines.hybrid_retrieval_pipeline)

context_hybrid_retrieval_pipeline = pipelines.retrieval_pipelines.hybrid_retrieval_pipeline.get_hybrid_retrieval_pipeline(context_indexing_store)

map_evaluator = DocumentMAPEvaluator()
mrr_evaluator = DocumentMRREvaluator()
recall_evaluator = DocumentRecallEvaluator()

retriever_to_evaluator_adapter = WrapInListAdapter()

context_hybrid_retrieval_pipeline.add_component(instance=map_evaluator, name="map_evaluator")
context_hybrid_retrieval_pipeline.add_component(instance=mrr_evaluator, name="mrr_evaluator")
context_hybrid_retrieval_pipeline.add_component(instance=recall_evaluator, name="recall_evaluator")
context_hybrid_retrieval_pipeline.add_component(instance=retriever_to_evaluator_adapter, name="retriever_to_evaluator_adapter")
context_hybrid_retrieval_pipeline.connect("joiner", "retriever_to_evaluator_adapter")
context_hybrid_retrieval_pipeline.connect("retriever_to_evaluator_adapter.output", "map_evaluator.retrieved_documents")
context_hybrid_retrieval_pipeline.connect("retriever_to_evaluator_adapter.output", "mrr_evaluator.retrieved_documents")
context_hybrid_retrieval_pipeline.connect("retriever_to_evaluator_adapter.output", "recall_evaluator.retrieved_documents")

🚅 Components
  - query_embedder: SentenceTransformersTextEmbedder
  - bm25_retriever: InMemoryBM25Retriever
  - retriever: InMemoryEmbeddingRetriever
  - joiner: DocumentJoiner
  - map_evaluator: DocumentMAPEvaluator
  - mrr_evaluator: DocumentMRREvaluator
  - recall_evaluator: DocumentRecallEvaluator
  - retriever_to_evaluator_adapter: WrapInListAdapter
🛤️ Connections
  - query_embedder.embedding -> retriever.query_embedding (List[float])
  - bm25_retriever.documents -> joiner.documents (List[Document])
  - retriever.documents -> joiner.documents (List[Document])
  - joiner.documents -> retriever_to_evaluator_adapter.documents (List[Document])
  - retriever_to_evaluator_adapter.output -> map_evaluator.retrieved_documents (List[List[Document]])
  - retriever_to_evaluator_adapter.output -> mrr_evaluator.retrieved_documents (List[List[Document]])
  - retriever_to_evaluator_adapter.output -> recall_evaluator.retrieved_documents (List[List[Document]])

## Run Evaluation

In [47]:
test_cases = [
    {
        "name": "Basic RAG",
        "pipeline": base_retrieval_pipeline,
    },
    {
        "name": "Hybrid RAG",
        "pipeline": hybrid_retrieval_pipeline,
    },
    {
        "name": "Contextual RAG",
        "pipeline": context_retrieval_pipeline,
    },
    {
        "name": "Contextual Hybrid RAG",
        "pipeline": context_hybrid_retrieval_pipeline,
    },
]

In [48]:
import numpy as np
from scipy.stats import norm

def _has_ci_converged(measurements, tol, alpha=0.05):
    """
    Return True if the (1-alpha)% confidence‐interval half‐width
    of the mean is <= tol.
    """
    n = len(measurements)
    if n < 2:
        return False
    mean = np.mean(measurements)
    var  = np.var(measurements, ddof=1)
    se = np.sqrt(var / n)
    z = norm.ppf(1 - alpha/2)
    half_width = z * se
    logging.info(
        "n=%d, mean=%.4f, var=%.6f, SE=%.6f, z=%.3f → half_width=%.6f (tol=%.6f)",
        n, mean, var, se, z, half_width, tol
    )
    return half_width <= tol

def did_converge(precisions, recalls, tolerance=0.01, alpha=0.05):
    """
    Decide whether to stop based on the CI half-width
    of both precision and recall.

    :param precisions: List[float]
    :param recalls:    List[float]
    :param step_size:  (unused) kept for API compatibility
    :param tolerance:  float, max allowable CI half-width
    :param alpha:      float, significance level (default 0.05 ⇒ 95% CI)
    :return:           bool, True if both metrics have CI half-width <= tolerance
    """
    return (_has_ci_converged(precisions, tolerance, alpha) and
            _has_ci_converged(recalls,    tolerance, alpha))

In [49]:
import logging

# silence haystack’s pipeline logs
logging.getLogger("haystack").setLevel(logging.WARNING)
logging.getLogger("haystack.core.pipeline").setLevel(logging.WARNING)

# if you see similar spam from transformers, ragas, etc.
logging.getLogger("transformers").setLevel(logging.WARNING)
logging.getLogger("ragas").setLevel(logging.WARNING)

In [ ]:
import config.prompt
importlib.reload(config.prompt)

results = []

# 1) Filter out the null‐question rows
df_nonnull = df[df["question"].notnull()]

df_shuffled = df_nonnull.sample(n=50, random_state=42).reset_index(drop=True)

# 3) Now iterate

for index, row in tqdm(
    df_shuffled.iterrows(),
    total=len(df_shuffled),
    desc="Processing rows",
    unit="row"
):
    reference = row["groundTruth"]
    relevant_documents = row["documents"]
    question = row["question"]

    for test_case in test_cases:
        pipeline = test_case["pipeline"]
        request_payload = {
            "query_embedder": {
                "text": question,
            },
            "map_evaluator": {
                "ground_truth_documents": [relevant_documents],
            },
            "mrr_evaluator": {
                "ground_truth_documents": [relevant_documents],
            },
            "recall_evaluator": {
                "ground_truth_documents": [relevant_documents],
            }
        }
        if "bm25_retriever" in pipeline.graph.nodes:
            request_payload["bm25_retriever"] = {
                "query": question
            }
        result = pipeline.run(request_payload)

        map_score = result.get("map_evaluator", {}).get("score", {})
        mrr_score = result.get("mrr_evaluator", {}).get("score", {})
        recall_score = result.get("recall_evaluator", {}).get("score", {})

        df.at[index, f"{test_case['name']}_map"] = map_score
        df.at[index, f"{test_case['name']}_mrr"] = mrr_score
        df.at[index, f"{test_case['name']}_recall"] = recall_score

Processing rows:   0%|          | 0/50 [00:00<?, ?row/s]

[[Document(id=32a47a42b8dbbbe94e2752cd1b2621e6f845383bc8dbb4d686b57f1ff906fe2b, content: 'Frauen mit Rhesus-negativem Blut können ebenfalls eine äußere Wendung durchführen lassen. Da dabei k...', meta: {'title': 'Äußere Wendung.md', 'source_id': '7ae1e3f0-dd60-4371-ab7b-aeeebef45f64', 'page_number': 1, 'split_id': 20, 'split_idx_start': 7638})]]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

10:57:21 WARNING No Documents found with embeddings. Returning empty list. To generate embeddings, use a DocumentEmbedder.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

10:57:21 WARNING No Documents found with embeddings. Returning empty list. To generate embeddings, use a DocumentEmbedder.
Processing rows: 100%|██████████| 50/50 [00:19<00:00,  2.62row/s]


In [51]:
embedding_model_name = os.environ['EMBEDDING_MODEL_NAME'].replace('/', '')
df.to_pickle(f"results/retrieval_eval_results_{embedding_model_name}_{os.environ['EMBEDDING_MODEL_TOP_K']}_topk{os.environ['EMBEDDING_MODEL_TOP_K']}.pkl")

In [54]:
df

,question,references,groundTruth,source_file,documents,variant,prev_messages,Basic RAG_map,Basic RAG_mrr,Basic RAG_recall,Hybrid RAG_map,Hybrid RAG_mrr,Hybrid RAG_recall,Contextual RAG_map,Contextual RAG_mrr,Contextual RAG_recall,Contextual Hybrid RAG_map,Contextual Hybrid RAG_mrr,Contextual Hybrid RAG_recall
0,Wie läuft eine äußere Wendung ab und was passi...,[Eine äußere Wendung findet in der Regel in ei...,Eine äußere Wendung findet ambulant in einer K...,Äußere Wendung.md,[Document(id=ea0c518b7c51e474b09770e8db9709652...,default,[],0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Und wie lange dauert dieser Eingriff normalerw...,[Eine äußere Wendung findet in der Regel in ei...,Eine äußere Wendung findet ambulant in einer K...,Äußere Wendung.md,[Document(id=ea0c518b7c51e474b09770e8db9709652...,contextual,"[Wie läuft eine äußere Wendung ab?, Bei der äu...",0.833333,1.0,1.0,0.333333,0.333333,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Ey, wie machen die eigentlich so 'ne äußere We...",[Eine äußere Wendung findet in der Regel in ei...,Eine äußere Wendung findet ambulant in einer K...,Äußere Wendung.md,[Document(id=ea0c518b7c51e474b09770e8db9709652...,slang,[],1.000000,1.0,1.0,0.333333,0.333333,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,How is an external version done and what happe...,[Eine äußere Wendung findet in der Regel in ei...,Eine äußere Wendung findet ambulant in einer K...,Äußere Wendung.md,[Document(id=ea0c518b7c51e474b09770e8db9709652...,english,[],0.500000,0.5,1.0,0.166667,0.166667,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Können Sie mir erklären, wie eine äußere Wendu...",[Eine äußere Wendung findet in der Regel in ei...,Eine äußere Wendung findet ambulant in einer K...,Äußere Wendung.md,[Document(id=ea0c518b7c51e474b09770e8db9709652...,german-variation-1,[],1.000000,1.0,1.0,0.333333,0.333333,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,"Was ist zu tun, wenn nach der Geburtseinleitun...",[Bei jeder medikamentösen oder mechanischen Me...,Sollten nach der Geburtseinleitung starke oder...,Einleitung der Geburt.md,[Document(id=8a64407f7a87fa95d3ce2f268e6b00c66...,german-variation-2,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
606,"Kann mir jemand sagen, was ich machen soll, we...",[Bei jeder medikamentösen oder mechanischen Me...,Sollten nach der Geburtseinleitung starke oder...,Einleitung der Geburt.md,[Document(id=8a64407f7a87fa95d3ce2f268e6b00c66...,german-variation-3,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,Ich hatte eine Geburtseinleitung – was mache i...,[Bei jeder medikamentösen oder mechanischen Me...,Sollten nach der Geburtseinleitung starke oder...,Einleitung der Geburt.md,[Document(id=8a64407f7a87fa95d3ce2f268e6b00c66...,german-variation-4,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,"Was empfehlen Sie, wenn nach der Einleitung de...",[Bei jeder medikamentösen oder mechanischen Me...,Sollten nach der Geburtseinleitung starke oder...,Einleitung der Geburt.md,[Document(id=8a64407f7a87fa95d3ce2f268e6b00c66...,german-variation-5,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
